In [4]:
!pip3 install langchain
!pip3 install pypdf
!pip3 install OpenAI
!pip3 install pypdf
!pip3 install rapidocr-onnxruntime
!pip3 install sentence-transformers
!pip3 install chroma
!pip3 install chromadb
!pip3 install tiktoken
!pip3 install llama-cpp-python
!pip3 install jq
# !pip3 install pdfminer.six
# !pip3 install Cython


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app

In [2]:
from openai import OpenAI
from langchain.llms import (HuggingFaceHub, LlamaCpp, OpenAI)
from langchain.document_loaders import (PyPDFLoader, JSONLoader)
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from langchain.retrievers.merger_retriever import MergerRetriever
import sys
import os
from langchain.agents import AgentType, Tool, initialize_agent

from dotenv import load_dotenv
import os

load_dotenv()

In [20]:
llm = OpenAI(temperature=0, api_key=os.getenv("OPENAI_API_KEY"))

# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# llm = LlamaCpp(
#     model_path="./mistral-7b-instruct-v0.2.Q4_K_M.gguf",
#     temperature=0.75,
#     max_tokens=2000,
#     top_p=1,
#     callback_manager=callback_manager,
#     verbose=True,  # Verbose is required to pass to the callback manager
# )


In [28]:

embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

chunks = text_splitter.split_text("""
    13. The undertaking shall disclose its transition plan for climate change mitigation.1 \n\n 
    14. The objective of this Disclosure Requirement is to enable an understanding of the 
    undertaking’s past, current, and future mitigation efforts  to  ensure  that  its  strategy  and 
    business model(s) are compatible with the transition to a sustainable economy, and with the 
    limiting of global warming to 1.5 °C in line with the Paris Agreement and with the objective of 
    achieving climate neutrality by 2050 and, where relevant, the undertaking’s exposure to coal, 
    oil and gas-related activities. \n\n
    15. The information required by paragraph 13 shall include:  \n\n
    (a) by reference to GHG emission reduction targets (as required by Disclosure 
    Requirement E1-4), an explanation of how the undertaking’s targets are compatible 
    with the limiting of global warming to 1.5°C in line with the Paris Agreement;\n\n
    (b) by reference to GHG emission reduction targets (as required by Disclosure 
    Requirement  E1-4)  and  the  climate  change  mitigation  actions  (as  required  by 
    Disclosure Requirement E1-3), an explanation of the decarbonisation levers 
    identified, and key actions planned, including  changes in the undertaking’s product 
    and service portfolio and its adoption of new technologies;  \n\n
    (c) by  reference  to  the  climate  change  mitigation  actions  (as  required  by  Disclosure 
    Requirement  E1-3), an explanation of the undertaking’s investments and funding 
    supporting the implementation of the transition plan;  \n\n
    (d) a qualitative assessment of the potential locked-in GHG emissions from the 
    undertaking’s key assets and products. This shall include an explanation of if and 
    how  these  emissions  may  jeopardise  the  achievement of the undertaking’s GHG 
    emission reduction targets and drive transition risk, and if applicable, an explanation 
    of the undertaking’s plans to manage its GHG-intensive and energy-intensive assets 
    and products;  \n\n
    (e) if applicable, an explanation of the undertaking’s objective for aligning its economic 
    activities  (revenues)  with  the  Taxonomy  Regulation  (EU)  2020/852  including  any 
    delegated  regulations  related  to  climate  change  mitigation  and  adaptation  and  its 
    plans for future Taxonomy alignment (revenues, CapEx and CapEx plans);\n\n
    (f) a disclosure on whether or not the undertaking is excluded from the EU Paris-aligned 
    Benchmarks;2  \n\n
    (g) an  explanation  of  how  the  transition  plan  is  embedded  in  and  aligned  with  the 
    undertaking’s  overall  business  strategy  and  financial  planning  and  whether  it  is 
    approved by its administrative, management and supervisory bodies; and \n\n
    (h) an explanation of the undertaking’s progress in implementing the transition plan. \n\n
    16. In case the undertaking does not have a transition plan in place, it shall indicate whether and, 
    if so, when it will adopt a transition plan.
""")

esrs1_store = Chroma.from_texts(chunks, embeddings, collection_name="Disclosure_Requirement_E1-1")
esrs1_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=esrs1_store.as_retriever())


In [23]:
pdfLoader = PyPDFLoader("./SUSTAINABILITY_REPORT_2022-1-12.pdf")

pages = pdfLoader.load()
docs = text_splitter.split_documents(pages)
report_store = Chroma.from_documents(docs[0:10], embeddings)

report_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=report_store.as_retriever())


In [29]:
tools = [
    Tool(
        name="ESRS E1-1",
        func=esrs1_chain.run,
        description="Describes Disclosure requirement E1-1 for sustainability reports. All sustainability reports should meet this requirement. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="O'Neil Sustainability report",
        func=report_chain.run,
        description="O'Neil's sustainability report from 2022. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [34]:
agent.run("Does O'Neill's 2022 sustainability report meet Disclosure Requirement 15d in the disclousre requirement E1-1? Cite the corresponding section from O'Neil's sustainability report.")




> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [5]:
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["chapterTitle"] = record.get("chapterTitle")
    metadata["reference"] = record.get("reference")
    return metadata


data = JSONLoader(
    file_path='./ESRS_E1.json',
    jq_schema='.requirements[]',
    content_key="content",
    metadata_func=metadata_func
).load()

print(data)

[Document(page_content='The objective of this Standard is to specify Disclosure Requirements which will enable users of sustainability statements to understand:', metadata={'source': '/Users/tzethoven/Documents/palau-project/PalauReportAnalyser/ESRS_E1.json', 'seq_num': 1, 'chapterTitle': 'Objective', 'reference': '1'}), Document(page_content='how the undertaking affects climate change, in terms of material positive and negative actual and potential impacts;', metadata={'source': '/Users/tzethoven/Documents/palau-project/PalauReportAnalyser/ESRS_E1.json', 'seq_num': 2, 'chapterTitle': 'Objective', 'reference': '1a'}), Document(page_content='the  undertaking’s  past,  current,  and  future  mitigation  efforts  in  line  with  the  Paris Agreement (or an updated international agreement on climate change) and compatible with limiting global warming to 1.5°C;', metadata={'source': '/Users/tzethoven/Documents/palau-project/PalauReportAnalyser/ESRS_E1.json', 'seq_num': 3, 'chapterTitle': 'O